In [108]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import torchvision
import torchvision.datasets as dsets
import torchvision.transforms as transforms

import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'svg'

In [6]:
use_cuda = True
device = torch.device('cuda:4' if use_cuda else 'cpu')

# 1、data set and data loader

In [ ]:
class housePriceDataSet(torch.utils.data.Dataset):
    def __init__(self):
        super(housePriceDataSet, self).__init__()
        self.train = pd.read_csv('data/housePrice/train.csv')
        
    def cls2idx(self, data):
        MSSbClassDict = {20:[0,0,0,0]}

In [71]:
d = pd.read_csv('data/housePrice/train.csv',)
dd = pd.read_csv('data/housePrice/test.csv',)
d = pd.concat((d,dd),ignore_index = True)
d

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500.0
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500.0
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500.0
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000.0
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2914,2915,160,RM,21.0,1936,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,6,2006,WD,Normal,NaN
2915,2916,160,RM,21.0,1894,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2006,WD,Abnorml,NaN
2916,2917,20,RL,160.0,20000,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2006,WD,Abnorml,NaN
2917,2918,85,RL,62.0,10441,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,Shed,700,7,2006,WD,Normal,NaN


In [106]:
mydict = {} # 形式
for cls in list(d)[1:-1]:
    if d[cls].to_numpy().dtype != 'O':
        continue
    c = set(d[cls].to_numpy())
    c = sorted([str(i) for i in c])
    l = len(c)
    a = {c[i]:i for i in range(l)}
    mydict.update({cls:a})

In [105]:
d['MSZoning'].to_numpy().dtype

dtype('O')

In [112]:
with open('data/housePrice/cls2idx.json','w') as f:
    json.dump(mydict,f)

In [84]:
c = set(d['MSSubClass'].to_numpy())
c = sorted([str(i) for i in c])
l = len(c)
a = {c[i]:i for i in range(l)}
a

{'120': 0,
 '150': 1,
 '160': 2,
 '180': 3,
 '190': 4,
 '20': 5,
 '30': 6,
 '40': 7,
 '45': 8,
 '50': 9,
 '60': 10,
 '70': 11,
 '75': 12,
 '80': 13,
 '85': 14,
 '90': 15}

In [80]:
c = set(d['MSZoning'].to_numpy())
c = sorted([str(i) for i in c])
l = len(c)
{c[i]:i for i in range(l)}

{'C (all)': 0, 'FV': 1, 'RH': 2, 'RL': 3, 'RM': 4, 'nan': 5}

['nan', 'RL', 'RH', 'C (all)', 'FV', 'RM']

In [85]:
d.MSSubClass.replace(a)

0        60
1        20
2        60
3        70
4        60
       ... 
2914    160
2915    160
2916     20
2917     85
2918     60
Name: MSSubClass, Length: 2919, dtype: int64

In [39]:
list(d)

['Id',
 'MSSubClass',
 'MSZoning',
 'LotFrontage',
 'LotArea',
 'Street',
 'Alley',
 'LotShape',
 'LandContour',
 'Utilities',
 'LotConfig',
 'LandSlope',
 'Neighborhood',
 'Condition1',
 'Condition2',
 'BldgType',
 'HouseStyle',
 'OverallQual',
 'OverallCond',
 'YearBuilt',
 'YearRemodAdd',
 'RoofStyle',
 'RoofMatl',
 'Exterior1st',
 'Exterior2nd',
 'MasVnrType',
 'MasVnrArea',
 'ExterQual',
 'ExterCond',
 'Foundation',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinSF1',
 'BsmtFinType2',
 'BsmtFinSF2',
 'BsmtUnfSF',
 'TotalBsmtSF',
 'Heating',
 'HeatingQC',
 'CentralAir',
 'Electrical',
 '1stFlrSF',
 '2ndFlrSF',
 'LowQualFinSF',
 'GrLivArea',
 'BsmtFullBath',
 'BsmtHalfBath',
 'FullBath',
 'HalfBath',
 'BedroomAbvGr',
 'KitchenAbvGr',
 'KitchenQual',
 'TotRmsAbvGrd',
 'Functional',
 'Fireplaces',
 'FireplaceQu',
 'GarageType',
 'GarageYrBlt',
 'GarageFinish',
 'GarageCars',
 'GarageArea',
 'GarageQual',
 'GarageCond',
 'PavedDrive',
 'WoodDeckSF',
 'OpenPorchSF'

In [26]:
d.to_numpy()[:,1:]

array([[60, 'RL', 65.0, ..., 'WD', 'Normal', 208500],
       [20, 'RL', 80.0, ..., 'WD', 'Normal', 181500],
       [60, 'RL', 68.0, ..., 'WD', 'Normal', 223500],
       ...,
       [70, 'RL', 66.0, ..., 'WD', 'Normal', 266500],
       [20, 'RL', 68.0, ..., 'WD', 'Normal', 142125],
       [20, 'RL', 75.0, ..., 'WD', 'Normal', 147500]], dtype=object)

In [19]:
d.MSSubClass

0       60
1       20
2       60
3       70
4       60
        ..
1455    60
1456    20
1457    70
1458    20
1459    20
Name: MSSubClass, Length: 1460, dtype: int64

In [ ]:
pd.DataFrame.get

In [37]:
pd.Series.

In [4]:
np.log2(1e6)

19.931568569324174

In [7]:
1460*0.8

1168.0